# IR rangefinder circuit test

This code tests functionality of a USB interface IR rangefinder circuit that I designed for testing bird preferences. The device connects to a PC via USB as a 115k Baud serial port. It reads analog range data from two IR sensors and outputs the range values when up or down thresholds are exceeded (that is, it acts as a switch sensor that also outputs range info). Up/down threshold levels, which determine the trigger distance, are set by sending serial text commands to the device. The device also has four LED outputs, two for each IR sensor, which can be turned on or off via serial commands.

#### IR sensor output looks like this:
- "aaaa,bbbb\r\n"
    - where a and b are integer values indicating IR level for sensors 1 & 2. Higher values = closer object

#### This is the help message printed if you type 'H' in a terminal window connected to the device:

<code>Help for operant device #6:
  H = get this help message
  N = get this device serial number
  I = get current IR sensor levels
  L,n,n,n,n = set LEDs 0=off,1=on, order: P1-L1, P1-L2, P2-L1, P2-L2
  D,n = set down (high to low) IR trigger threshold
  U,n = set up (low to high) IR trigger threshold
  T = enter test mode
  E = enter experiment mode (default)
</code>

Note: when sending a command, you must end the string w/ "\r\n"



## Helper functions

In [1]:
import time 
import serial

def send_IR_command(ser, cmd):
    """Send a command to the IR proximity switch"""
    ser.write(('%s\r\n'%(cmd)).encode('utf-8'))
    
def set_LED_state(ser, l1a, l1b, l2a, l2b):
    """Set IR proximity switch LED states"""
    send_IR_command(ser, 'L,%d,%d,%d,%d'%(l1a,l1b,l2a,l2b))


## Test the LEDs

Note: The device I'm using doesn't turn on/off LEDs P1-L1 or P2-L1. Not sure why, but they aren't needed for my project anyway.


In [16]:
import serial
import time 

baudrate = 115200
port = 'COM5'

# in case the port was left open, try to close it
try:
    ser.close()
except:
    pass

ser = serial.Serial(port, baudrate, timeout=0)

# note that the first and third LEDs aren't working on 
#  the device I'm using here
set_LED_state(ser,0,1,0,1)
time.sleep(2)

set_LED_state(ser,0,0,0,1)
time.sleep(2)

set_LED_state(ser,0,1,0,0)
time.sleep(2)

set_LED_state(ser,0,0,0,0)
time.sleep(2)

ser.close()

## Set device to test mode

In test mode, LED turns on when switch is high (on) and off when low(off). This works for both IR rangefinders.

In [12]:
def send_IR_command(ser, cmd):
    ser.write(('%s\r\n'%(cmd)).encode('utf-8'))
    
try:
    ser.close()
except:
    pass

ser = serial.Serial(port, baudrate, timeout=0)
# ser.write('t\r\n'.encode('utf-8'))
send_IR_command(ser,'t')
ser.close()

## Set device to experiment mode (default)

In experiment mode, the LEDs aren't linked to the switch levels.

In [13]:
try:
    ser.close()
except:
    pass

ser = serial.Serial(port, baudrate, timeout=0)
# ser.write('e\r\n'.encode('utf-8'))
send_IR_command(ser,'e')
ser.close()

## Switch sensor loop

Baiscally replicate the device test mode in software: 
- LED on if sensor above threshold, off if below

#### Code comments:


- Periodically requests device to send current IR levels. This is necessary because sometimes a threshold state transition is not reported. To catch this, I poll periodically.


- There is software threshold logic on top of the hardware threshold detection. This is used to reduce the tendency for the device to send out many up/down transition detections when an object is at threshold. So in software, you should only see few on/off triggerings.

In [15]:
import serial
import time 

def parse_proximity_levels(instr):
    """Convert 'nn,nn' string to int state levels """
    try:
        s1 = int(instr.split(',')[0])
        s2 = int(instr.split(',')[1])
        return s1,s2
    except:
        return 0,0
    
def get_switch_state(oldstate, on_thresh, off_thresh, instr):
    """determine current switch state,
      return whether state has changed, and current state"""
    # convert "num,num" str into int values
    s1,s2 = parse_proximity_levels(instr)
    # bypass if error parsing
    if (s1!=0) & (s2!=0):
        newstate = oldstate.copy()
        if s1 > on_thresh: newstate[0] = 1
        elif s1 < off_thresh: newstate[0] = 0
        if s2 > on_thresh: newstate[1] = 1
        elif s2 < off_thresh: newstate[1] = 0
        return newstate!=oldstate, newstate
    return False, oldstate

def check_IR_switch(ser, sw_state, on_thresh, off_thresh):
    """Poll port for device output, process it & return switch state"""
    error = False
    changed = False
    # if incoming bytes are waiting to be read from the 
    #  serial input buffer
    if ser.in_waiting > 0: 
        #read the bytes and convert from binary array to ASCII
        while ser.in_waiting > 0:
            data_str = ser.readline().decode('ascii') 
        # update switch on/off state, check for change
        changed, newstate = get_switch_state(sw_state, 
                                         on_thresh, 
                                         off_thresh,
                                         data_str)
        # update switch state
        if changed == True:
            # sync LED state w/ switch state
            # set_LED_state(ser, 0, newstate[0], 0, newstate[1])
            sw_state = newstate
                
    return error, changed, sw_state 

# IR switch configs
IR_threshold = 400 # hardware on/off threshold
on_thresh = IR_threshold + 25 # software on threshold
off_thresh =  IR_threshold - 25 # software off threshold
check_time = 0.5 # periodic IR level check interval

# state variables
lastcheck = 0
sw_state = [0,0] 

# make sure serial port of closed before trying to open it
try:
    ser.close()
except:
    pass

# connect to device serial port
#  Note: COM port # will be different on different PCs
baudrate = 115200
port = 'COM5'
ser = serial.Serial(port, baudrate, timeout=0)

# set device on / off switch thresholds
send_IR_command(ser, 'u,%d'%(IR_threshold))
send_IR_command(ser, 'd,%d'%(IR_threshold))

quit = False
while not quit:
    try:
        # check switch device, handle message, return state
        quit, changed, sw_state = check_IR_switch(
            ser, sw_state, on_thresh, off_thresh)
        
        # set LED to current switch state
        if changed == True:
            changed = False
            set_LED_state(ser,0,sw_state[0],0,sw_state[1])

        # switch is on/triggered
        if sw_state[trigger_switch] == 1:
            last_on_time = time.time()

        # trigger an IR level report every check_time seconds.
        #  This catches occasional missed switch threshold changes.
        if time.time() > lastcheck + check_time:
            send_IR_command(ser, 'i')
            lastcheck = time.time()

        time.sleep(0.01) 
        
    # keep track of video here: check if done playing
    except:
        quit = True
        
print('\nStopped monitoring port')
ser.close()



Stopped monitoring port


In [180]:
# this is here in case you need to close the port manually
ser.close()